In [1]:
## ACORN fly tracking script
## Contains functions that
## 1) sequentially open images or videos saved in a specified location and determines the centroid location
## of the largest spatial cluster in the background-subtracted image

# Dependencies
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imutils
import os
import time
import datetime
import skimage

In [4]:
def TrackVideo(fileLocation, saveLocation="/home/pi/Desktop/Tracking/", blur_kernel=(21,21), minArea= 250, maxArea=7500, showTracking=1, captureTracking=0):

    centroids = np.zeros((1,2))
    fgbg = cv2.createBackgroundSubtractorMOG2()
    capture = cv2.VideoCapture(fileLocation)
    capture.open(fileLocation);
    
    if captureTracking == 1:
        out = cv2.VideoWriter('Tracking.avi',cv2.VideoWriter_fourcc(*'MPEG'), 60, (1300,960), isColor=False)
    
    while(capture.isOpened()):
        [readokay, frame] = capture.read()
        if not readokay:
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if showTracking == 1:
            original = gray.copy()
            track_path = gray.copy()
        gray = cv2.GaussianBlur(gray, blur_kernel, 0)
        if showTracking == 1:
            contour_img = gray.copy()

        fgmask = fgbg.apply(gray)
        fgmask = cv2.dilate(fgmask, None, iterations=3)
        
        # Remove shadows by thresholding
        thresh = cv2.threshold(fgmask, 127, 255, cv2.THRESH_BINARY)[1]
        
        # Find contours
        cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        contour_array = np.zeros([len(cnts),3])
        contour_id = 0
        for c in cnts:
        # Only keep contours that are within designated size
            if cv2.contourArea(c) > minArea and cv2.contourArea(c) < maxArea:
                M = cv2.moments(c)
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                contour_array[contour_id,0], contour_array[contour_id,1] = cX, cY
                contour_array[contour_id,2] = cv2.contourArea(c)
                contour_id += 1
                
                if showTracking == 1:
                    cv2.drawContours(contour_img, [c], -1, (0, 255, 0), thickness=cv2.FILLED)
                    cv2.circle(contour_img, (cX, cY), 7, (255, 255, 255), -1)
                    
        # Decide on best contour based on size and distance from last best contour
        if contour_array.size and any(contour_array[:,2] !=0):
            contour_max_x = int( contour_array[contour_array[:,2]==np.nanmax(contour_array[:,2]), 0 ] )
            contour_max_y = int( contour_array[contour_array[:,2]==np.nanmax(contour_array[:,2]), 1 ] )
            centroids = np.vstack([ centroids, [contour_max_x, contour_max_y] ])
            

        if showTracking == 1:
            
            for points in range(len(centroids[:,0])):
                cv2.circle(track_path, ( int( centroids[points,0] ), int( centroids[points,1] ) ), 7, (255, 255, 255), -1)
                
            trackView1 = np.concatenate( (cv2.resize(original, (650,480)), cv2.resize(thresh, (650,480))), axis=1 )
            trackView2 = np.concatenate( (cv2.resize(contour_img, (650,480)), cv2.resize(track_path, (650,480))), axis=1 )
            trackView = np.concatenate( (trackView1, trackView2), axis=0 )
            trackView = cv2.putText(trackView, "Original", (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1, cv2.LINE_AA)
            trackView = cv2.putText(trackView, "Background subtraction", (675,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,255), 1, cv2.LINE_AA)
            trackView = cv2.putText(trackView, "Contours", (25,505), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1, cv2.LINE_AA)
            trackView = cv2.putText(trackView, "Tracking", (675,505), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1, cv2.LINE_AA)
            
            if captureTracking == 1:
                out.write(trackView)
                
            cv2.imshow('Tracking View', trackView)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    if showTracking == 1:      
        capture.release()
    if captureTracking == 1:
        out.release()
    cv2.destroyAllWindows()
    return

In [5]:
TrackVideo(fileLocation="C:/Users/alisc/Desktop/Videos/vid2.h264", saveLocation="C:/Users/alisc/Desktop/Videos/", captureTracking=0)